In [4]:
import csv
import os
from kafka import KafkaProducer

# กำหนด Kafka Broker และ Topic
KAFKA_BROKER = '127.0.0.1:9092'
TOPIC_NAME = 'csv_topic'

# สร้าง Kafka Producer
producer = KafkaProducer(bootstrap_servers=[KAFKA_BROKER])

# กำหนดขีดจำกัดของขนาดฟิลด์ที่ csv จะอ่านได้
csv.field_size_limit(1000 * 1024 * 1024) 

def process_csv(file_path):
    print(f"Processing file: {file_path}")
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            key = row.get('Title', 'default_key')  # ใช้คอลัมน์ 'Title' เป็น key หรือกำหนดค่า default
            value = '฿'.join([str(row[col]) for col in reader.fieldnames])  # แปลงข้อมูลทั้งหมดเป็น string

            # ส่งข้อมูลไปยัง Kafka
            producer.send(
                TOPIC_NAME,
                key=str(key).encode('utf-8'),
                value=value.encode('utf-8')
            )

# รายชื่อไฟล์ CSV ที่ต้องการประมวลผล
csv_files = ['../ExtractedData/2018.csv','../ExtractedData/2019.csv','../ExtractedData/2020.csv','../ExtractedData/2021.csv','../ExtractedData/2022.csv','../ExtractedData/2023.csv',
             '../scopusNewData/scopus_papers_2018_full_details.csv','../scopusNewData/scopus_papers_2019_full_details.csv','../scopusNewData/scopus_papers_2020_full_details.csv',
             '../scopusNewData/scopus_papers_2021_full_details.csv', '../scopusNewData/scopus_papers_2022_full_details.csv', '../scopusNewData/scopus_papers_2023_full_details.csv',
             '../scopusNewData/scopus_papers_2024_full_details.csv']

# ตรวจสอบว่าไฟล์มีอยู่จริงและประมวลผลแต่ละไฟล์
for csv_file in csv_files:
    if os.path.exists(csv_file):
        process_csv(csv_file)
    else:
        print(f"File not found: {csv_file}")


Processing file: ../ExtractedData/2018.csv
Processing file: ../ExtractedData/2019.csv
Processing file: ../ExtractedData/2020.csv
Processing file: ../ExtractedData/2021.csv
Processing file: ../ExtractedData/2022.csv
Processing file: ../ExtractedData/2023.csv
Processing file: ../scopusNewData/scopus_papers_2018_full_details.csv
Processing file: ../scopusNewData/scopus_papers_2019_full_details.csv
Processing file: ../scopusNewData/scopus_papers_2020_full_details.csv
Processing file: ../scopusNewData/scopus_papers_2021_full_details.csv
Processing file: ../scopusNewData/scopus_papers_2022_full_details.csv
Processing file: ../scopusNewData/scopus_papers_2023_full_details.csv
Processing file: ../scopusNewData/scopus_papers_2024_full_details.csv


In [5]:
from kafka import KafkaConsumer

# กำหนด Kafka Broker และ Topic
KAFKA_BROKER = '127.0.0.1:9092'
TOPIC_NAME = 'csv_topic'

# กำหนด key ที่ต้องการกรองใน Consumer
TARGET_KEY = 'Embracing innovation and collaboration: A message from the new Editor-in-Chief'  # ตัวอย่าง key ที่ต้องการกรอง

# สร้าง Kafka Consumer
consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=[KAFKA_BROKER],
    group_id='csv_consumer_group',
    auto_offset_reset='earliest',  # อ่านข้อความตั้งแต่แรก
    value_deserializer=lambda x: x.decode('utf-8'),
    key_deserializer=lambda x: x.decode('utf-8')
)


def consume_messages():
    print("Starting consumer...")

    for message in consumer:
        # อ่าน key และ value จาก Kafka message
        key = message.key
        value = message.value

        # กรองข้อมูลตาม key ที่ต้องการ
        if key == TARGET_KEY:
            print(f"Consumed message with key: {key}, value: {value}")
            break



# เริ่มต้น Kafka Consumer
consume_messages()


Starting consumer...
Consumed message with key: Embracing innovation and collaboration: A message from the new Editor-in-Chief, value: Embracing innovation and collaboration: A message from the new Editor-in-Chief฿฿Li, Min฿Journal฿Cancer Letters฿2024-12-28฿University of Oklahoma College of Medicine฿
